In [1]:
import pandas as pd
import numpy as np
import theano.tensor as T
import theano
import theano.gradient
import pyipopt

In [2]:
inputfile = './data_new_volume.csv'
df = pd.read_csv(inputfile)
df['const'] = 1

df['pgmidgrade_km_adj'].fillna(value=10000, inplace=True)

df = df[df['treattype'] < 3]
for elem in df['treattype'].unique():
    df['treat' + str(elem)] = df['treattype'] == elem

for elem in df['choice'].unique():
    df['choice' + str(elem)] = df['choice'] == elem

df['treat1_topusage'] = df['treat1']*df['dv_usageveh_p75p100']
df['treat2_topusage'] = df['treat2']*df['dv_usageveh_p75p100']

df['treat1_college'] = df['treat1']*df['dv_somecollege']
df['treat2_college'] = df['treat2']*df['dv_somecollege']

df.head()

In [3]:
#df = df[ df['dv_carpriceadj_p75p100'] == 1]
#df = df[ df['dv_usageveh_p75p100'] == 0]
df.head()

In [4]:
price_labels = ['pg_km_adj', 'pgmidgrade_km_adj', 'pe_km_adj']
value_labels = ['value_total']

# all
Xexpd_labels = ['dv_ctb', 'dv_bh', 'dv_rec', 'dv_female', 'dv_age_25to40y', 'dv_age_morethan65y', 'dv_somesecondary', 'dv_somecollege', 'dv_carpriceadj_p75p100', 'dv_usageveh_p75p100', 'stationvisit_avgcarprice_adj', 'const']
Xutil_labels = ['dv_ctb', 'dv_bh', 'dv_rec', 'dv_female', 'dv_age_25to40y', 'dv_age_morethan65y', 'dv_somesecondary', 'dv_somecollege', 'dv_carpriceadj_p75p100', 'dv_usageveh_p75p100', 'stationvisit_avgcarprice_adj', 'const']

# usage
#Xexpd_labels = ['choice2', 'choice3', 'dv_ctb', 'dv_bh', 'dv_rec', 'dv_female', 'dv_age_25to40y', 'dv_age_morethan65y', 'dv_somesecondary', 'dv_somecollege', 'dv_carpriceadj_p75p100', 'stationvisit_avgcarprice_adj', 'const']
#Xutil_labels = ['dv_ctb', 'dv_bh', 'dv_rec', 'dv_female', 'dv_age_25to40y', 'dv_age_morethan65y', 'dv_somesecondary', 'dv_somecollege', 'dv_carpriceadj_p75p100', 'stationvisit_avgcarprice_adj', 'const']

# car price
#Xexpd_labels = ['dv_ctb', 'dv_bh', 'dv_rec', 'dv_female', 'dv_age_25to40y', 'dv_age_morethan65y', 'dv_somesecondary', 'dv_somecollege', 'dv_usageveh_p75p100', 'stationvisit_avgcarprice_adj', 'const']
#Xutil_labels = ['dv_ctb', 'dv_bh', 'dv_rec', 'dv_female', 'dv_age_25to40y', 'dv_age_morethan65y', 'dv_somesecondary', 'dv_somecollege', 'dv_usageveh_p75p100', 'stationvisit_avgcarprice_adj', 'const']

Xlelas_labels = ['const', 'dv_carpriceadj_p75p100', 'dv_somecollege', 'dv_usageveh_p75p100', 'treat1', 'treat2']
Xlsigma_labels = ['const']
Xlmu_labels = ['const']

In [5]:
floatX = 'float64'

In [6]:
choice = df.loc[:, 'choice'].as_matrix() - 1
price  = df.loc[:, price_labels].as_matrix().astype(floatX)
value  = df.loc[:, value_labels].as_matrix().astype(floatX)
Xexpd  = df.loc[:, Xexpd_labels].as_matrix().astype(floatX)
Xutil  = df.loc[:, Xutil_labels].as_matrix().astype(floatX)

Xlelas = df.loc[:, Xlelas_labels].as_matrix().astype(floatX)
Xlsigma = df.loc[:, Xlsigma_labels].as_matrix().astype(floatX)
Xlmu   = df.loc[:, Xlmu_labels].as_matrix().astype(floatX)

In [7]:
nobs, nchoice = price.shape
nobs, nXexpd = Xexpd.shape
nobs, nXutil = Xutil.shape

nobs, nXlelas = Xlelas.shape
nobs, nXlsigma = Xlsigma.shape
nobs, nXlmu = Xlmu.shape

In [115]:
chosenprice = price[range(nobs), choice].reshape(nobs,1)
dvchoice = (choice == np.arange(nchoice).reshape(nchoice,1)).transpose()
convenience_expend = 50.0
dvconvenience = np.abs(value.squeeze()-convenience_expend) < 1e-3
inconvenience, = np.where(~dvconvenience)

In [93]:
theta0 = np.zeros(nXlelas + nXlsigma + nXlmu + nXexpd + (nchoice-1)*nXutil)
theta0[0] = -0.1

In [118]:
def getparams(theta):
    offset = 0
    gammalelas = theta[offset:offset+nXlelas].reshape((nXlelas, 1))
    
    offset += nXlelas
    gammalsigma = theta[offset:offset+nXlsigma].reshape((nXlsigma, 1))
    
    offset += nXlsigma
    gammalmu = theta[offset:offset+nXlmu].reshape((nXlmu, 1))
    
    offset += nXlmu
    betaexpd = theta[offset:offset+nXexpd].reshape((nXexpd, 1))
    
    offset += nXexpd
    betautil = theta[offset:offset+(nchoice-1)*nXutil].reshape((nXutil, nchoice-1))
    
    offset += (nchoice-1)*nXutil
#     benefit = theta[offset]
    
    return gammalelas, gammalsigma, gammalmu, betaexpd, betautil, benefit

theta = T.dvector('theta')
gammalelas, gammalsigma, gammalmu, betaexpd, betautil, benefit = getparams(theta)

# priceT = theano.shared(price.astype(floatX), name='price')
# pchoiceT = theano.shared(pchoice.astype(floatX), name='pchoice')
# XexpdT = theano.shared(Xexpd.astype(floatX), name='Xexpd')
# XutilT = theano.shared(Xutil.astype(floatX), name='Xutil')
# valueT = theano.shared(value.astype(floatX), name='value')
# XlelasT = theano.shared(Xlelas.astype(floatX), name='Xlelas')
# XlsigmaT = theano.shared(Xlsigma.astype(floatX), name='Xlsigma')
# XlmuT = theano.shared(Xlmu.astype(floatX), name='Xlmu')

mu = T.exp(T.dot(Xlmu, gammalmu))
elas = T.exp(T.dot(Xlelas, gammalelas))
lsigma = T.dot(Xlsigma,gammalsigma)
rho = elas - 1

omegal = 1e-3
omegah = elas**(1.0+1.0/(elas-1))

for i in range(7):
    omegal = (1-elas)*(omegal**(1/elas) - 1 + benefit)/(omegal**(1/elas-1) - 1)
    omegah = (1-elas)*(omegah**(1/elas) - 1 + benefit)/(omegah**(1/elas-1) - 1)

eta = np.log(value) + rho*np.log(chosenprice) - T.dot(Xexpd,betaexpd)
lexpend = np.log(value) - rho*(np.log(price) - np.log(chosenprice))
util = (T.exp(lexpend)/rho + T.concatenate([np.zeros((nobs,1)), T.dot(Xutil, betautil)],axis=1))/mu
maxutil = T.max(util,axis=1,keepdims=True)
ldenom = maxutil.squeeze() + T.log(T.sum(T.exp(util - maxutil),axis=1))
lprobchoice = T.sum(util*dvchoice.astype(floatX),axis=1) - ldenom
lpdfeta = eta*eta/(2*T.exp(2*lsigma)) + lsigma + np.log(2*np.pi)/2
nlogl = -lprobchoice[inconvenience].sum() + lpdfeta[inconvenience].sum()

In [119]:
def buildfunc(theta, nloglf):
    return (theano.function([theta], nloglf),
            theano.function([theta], T.grad(nloglf, theta)),
            theano.function([theta], theano.gradient.hessian(nloglf, theta)))

eval_f, eval_grad, eval_hess = buildfunc(theta, nlogl)

In [120]:
eval_hess(theta0)

In [121]:
pyipopt.set_loglevel(1)
thetahat , _, _, _, _, fval = pyipopt.fmin_unconstrained(
    eval_f,
    theta0,
    fprime=eval_grad,
    fhess=eval_hess,
    )


In [122]:
covhat = np.linalg.inv(eval_hess(thetahat))
sehat = np.sqrt(np.diagonal(covhat))
t = thetahat/sehat

gammalelashat, gammalsigmahat, gammalmuhat, betaexpdhat, betautilhat,_ = getparams(thetahat)
gammalelasse, gammalsigmase, gammalmuse, betaexpdse, betautilse,_ = getparams(sehat)
gammalelast, gammalsigmat, gammalmut, betaexpdt, betautilt,_ = getparams(t)

formatstr = '%30s%10.3f%10.3f%10.3f'

print '-'*60

print ' \n*** ln(elas) equation'
for i in range(nXlelas):
    print formatstr % (Xlelas_labels[i], gammalelashat[i], gammalelasse[i], gammalelast[i])
print '-'*60

print ' \n*** ln(sigma) equation'
for i in range(nXlsigma):
    print formatstr % (Xlsigma_labels[i], gammalsigmahat[i], gammalsigmase[i], gammalsigmat[i])
print '-'*60

print ' \n*** ln(mu) equation'
for i in range(nXlmu):
    print formatstr % (Xlmu_labels[i], gammalmuhat[i], gammalmuse[i], gammalmut[i])
print '-'*60

print ' \n*** Expenditure equation'
for i in range(nXexpd):
    print formatstr % (Xexpd_labels[i], betaexpdhat[i], betaexpdse[i], betaexpdt[i])
print '-'*60
    
print ' \n*** Discrete choice equation'
for j in range(nchoice-1):
    print '-------- choice', j+1, '------------------------------------------'
    for i in range(nXutil):
        print formatstr % (Xutil_labels[i], betautilhat[i][j], betautilse[i][j], betautilt[i][j])
print '-'*60
